In [ ]:
import pandas as pd
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import cross_val_score

Read in data: 

In [ ]:
data_directory = "../input/"
data = pd.read_csv(data_directory + '/spam.csv', encoding='latin-1') 
data.head()

Generate tesing and trainning data set to 25% to 75%: 

In [ ]:
text_train, text_test, tag_train, tag_test = train_test_split(data.v2,data.v1, test_size=0.25,random_state=0) 
print(text_train.shape, text_test.shape,tag_train.shape,tag_test.shape)
print()
print("After Spliting:")
print(text_train.head())
print()
print(tag_train.head())

Vectorized Counts of words: 

    fit() : used for generating learning model parameters from training data

    transform() : parameters generated from fit() method,applied upon model to generate transformed data set.

    fit_transform() : combination of fit() and transform() api on same data set



In [ ]:
vectorizer = CountVectorizer()

text_train_count = vectorizer.fit_transform(text_train)
text_test_count  = vectorizer.transform(text_test)
# print ('Summary:')
# print (text_train_count.shape)
# print (text_test_count.shape)
# print (vectorizer.vocabulary_)


Generate Frequncy Table: 

np.sum([[0, 1], [0, 5]], axis=0) -->>
array([0, 6])

np.sum([[0, 1], [0, 5]], axis=1) -->>
array([1, 5])

In [ ]:
# Occurrences take data_train_count to become a matrix like 2d array, then use sum with axis = 0 to add value from each columns vertically to get an array that each contains the 
# total sume of each letter
wordFreqeuncy = pd.DataFrame({'Word': vectorizer.get_feature_names(), 'occurrences':text_train_count.toarray().sum(axis=0)})
# print(data_train_count.toarray().sum(axis=0))
wordFreqeuncy['frequency'] = wordFreqeuncy['occurrences']/np.sum(wordFreqeuncy['occurrences'])
plt.plot(wordFreqeuncy.occurrences) # plot occurance with occurance id
plt.show()


To sort the table based on occurrences from high to low :

In [ ]:
wordFreqeuncySort = wordFreqeuncy.sort_values(by=['occurrences'], ascending=False)
wordFreqeuncySort.head()

Trainning the Model Based on Naive Bayes algorithm: 
*        We use SkLearn's MultinomialNB model to implements NB Algorithm.

    Naive Bayes algorithm: (sources: https://www.analyticsvidhya.com/blog/2017/09/naive-bayes-explained/)
        It is a classification technique based on Bayes’ Theorem with an assumption of independence among predictors. In simple terms, a Naive Bayes classifier assumes that the presence of a particular feature in a class is unrelated to the presence of any other feature.
        Equation: 
            P(c | x) = P( x | c) * P(c) / P (x)
            
            P(c|x) is the posterior probability of class (c, target) given predictor (x, attributes).
            P(c) is the prior probability of class.
            P(x|c) is the likelihood which is the probability of predictor given class.
            P(x) is the prior probability of predictor.


In [ ]:
clf = MultinomialNB()
clf.fit(text_train_count, tag_train)
predictions = clf.predict(text_test_count)
print(predictions)

Evaluate The Model: 

In [ ]:
# Compare tag_test with our prediction: 
print ("accuracy_score: ",accuracy_score(tag_test, predictions))
print()
print("classification_report")
print (classification_report(tag_test, predictions))
print()
print("confusion_matrix")
print (confusion_matrix(tag_test, predictions))
print()
print("cross_val_score")
cross_val = cross_val_score(clf, text_train_count, tag_train, cv=20, scoring='accuracy')
print (cross_val)
print()
print("Mean of cross_val_score: ",np.mean(cross_val))

* accuracy_score is fiarly high for our model considers accuracy_score = Total number of correct prediction / Total number of prediction
* classification_report:
    * precision is the ability of a classiifer not to label an instance positive that is actually negative. “for all instances classified positive, what percent was correct?”, we gets a fairly high one.
    * Recall is the ability of a classifier to find all positive instances.“for all instances that were actually positive, what percent was classified correctly?”. We gets a very high score on Recall.
    * The F1 score is a weighted harmonic mean of precision and recall such that the best score is 1.0 and the worst is 0.0. As a rule of thumb, the weighted average of F1 should be used to compare classifier models, and we have a fairly high score.
    * Support is the number of actual occurrences of the class in the specified dataset.
* Confusion Matrix: 
    *   We have relative large number of prediction are matched with what we have
* Cross val score: 
    *   These scores gave us percentage of corectness for each cases.
    *   We take mean of all of them and get approximately 98% of our observations are correctly predicted. 
 